<a href="https://colab.research.google.com/github/suplab/amazon-bedrock-genai-labs/blob/main/06_Using_Foundation_Models_For_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Assessment and Efficiency

Let's explore the core mechanisms Amazon Bedrock provides for evaluating and optimizing AI model performance at scale. The material introduces several evaluation strategies including programmatic, model-based, human-led, and RAG-specific assessments.

Beyond evaluation, we highlights techniques for operational efficiency, such as prompt caching, intelligent prompt routing, and cross-region inference profiles.

These capabilities enable developers to streamline model responses, reduce latency, cut costs, and dynamically allocate resources, making Amazon Bedrock a powerful platform for scalable, high-performance AI deployment.



## Bedrock Evaluations

### Automatic/Programmatic Evaluation
- Tests metrics like toxicity, accuracy, and robustness
- Configurable parameters: temperature, top P, response length
- Supports different task types (text generation, summarization, Q&A, classification)

### Judge Model Evaluation
- Uses an evaluator model to generate metrics
- Metrics include quality, helpfulness, relevance, coherence, responsible AI
- Can evaluate external inferences via JSONL files

### Human Worker Evaluation
- Uses human evaluators instead of models
- Can compare responses from two models
- Supports rating methods like thumbs up/down or Likert scale

### RAG Evaluation
- Evaluates retrieval-augmented generation systems
- Two evaluation types: Retrieval + Response Generation, Retrieval Only
- Metrics vary by evaluation type

### Prompt Caching
- Purpose: Optimize token usage and response time for repeated prompt components
- Benefits: Reduces token consumption Improves response speed Lower costs
- Limitations: Requires exact match of prefix content Sensitive to formatting changes
- Implementation Example:
```python
 cache_point = {"type": "default"}
  prompt = {
    "system_prompt": "...",
    "cache_point": cache_point,
    "user_input": "..."
  }
```

### Intelligent Prompt Routing
- Purpose: Automatically direct requests to appropriate models
- Features:
  * Dynamic model selection based on request complexity
  * Cost optimization:
  * Supports model families (e.g., Meta, Anthropic)

- Implementation Example:
```python
 routing_config = {
    "models": ["<MODEL_ID_1>", "<MODEL_ID_2>"],
    "routing_criteria": {"response_quality_difference": <DIFFERENCE>},
    "fallback_model": "<FALLBACK_MODEL_ID>"
  }
```

### Cross-Region Inference Profiles
- Purpose: Optimize model execution across regions
- Benefits:
  * Improved latency
  * Better availability
  * Load balancing
- Can be combined with prompt routing for comprehensive optimization

# Prompt Caching with Amazon Bedrock

Let's explore the Amazon Bedrock Converse API through an interactive chat session. You will learn how to cache responses and conserve tokens to reduce the overall cost of using Bedrock.

## Setting up Bedrock

In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00


In [7]:
import boto3
import json
from datetime import datetime

bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

MODEL_ID = "amazon.nova-micro-v1:0"

## Training Bedrock

In [8]:
system_prompt = '''
 You are an assistant that summarizes music reviews for a record company.
 Here are examples:

 Review: The latest album by The New Wave Band is a masterpiece! Every track is a hit.
 Summary: Reviewer praises the latest album as a masterpiece with hit tracks.

 Review: I was disappointed with the new single; it lacked the energy of their previous work.
 Summary: Reviewer expresses disappointment, noting a lack of energy compared to previous work.
 '''

user_input_review = "This EP is a solid effort with a few standout songs, though some tracks feel repetitive."

This will tell Bedrock what it's going to be doing for you. It provides examples of customer reviews, and then sets it up with one that you'll be using in an upcoming step.

## No caching

In [ ]:
no_cache_payload = {
     "system": [
         {"text": system_prompt}
     ],
     "messages": [
         {
             "role": "user",
             "content": [
                 {"text": user_input_review + "\nSummary:"}
             ]
         }
     ]
 }

no_cache_response = bedrock.converse(
     modelId=MODEL_ID,
     system=no_cache_payload["system"],
     messages=no_cache_payload["messages"]
 )

no_cache_output = no_cache_response['output']['message']['content'][0]['text']
no_cache_input_tokens = no_cache_response['usage']['inputTokens']
no_cache_output_tokens = no_cache_response['usage']['outputTokens']
no_cache_tokens = no_cache_input_tokens + no_cache_output_tokens

print("[No Caching] Generated Summary:")
print(no_cache_output)
print(f"Total Tokens Used (No Cache): {no_cache_tokens}")

The response should look similar to the following:

```
[No Caching] Generated Summary:
Reviewer acknowledges the EP as a solid effort with a few standout songs, but notes that some tracks feel repetitive.
Total Tokens Used (No Cache): 130
```

## With caching

In [ ]:
cache_point = {"cachePoint": {"type": "default"}}

payload_with_caching = {
     "system": [
         {"text": system_prompt},
         cache_point
     ],
     "messages": [
         {
             "role": "user",
             "content": [
                 {"text": user_input_review + "\nSummary:"}
             ]
         }
     ]
 }

response = bedrock.converse(
     modelId=MODEL_ID,
     system=payload_with_caching["system"],
     messages=payload_with_caching["messages"]
 )

cache_output = response['output']['message']['content'][0]['text']
cache_input_tokens = response['usage']['inputTokens']
cache_output_tokens = response['usage']['outputTokens']
cache_tokens = cache_input_tokens + cache_output_tokens

print("[With Caching] Generated Summary:")
print(cache_output)
print(f"Total Tokens Used: {cache_tokens}")

The response should look similar to the following:

```
 [With Caching] Generated Summary:
 Reviewer acknowledges the EP as a solid effort with a few standout songs, but notes that some tracks feel repetitive.
 Total Tokens Used: 46
```
Notice the dramatic decrease in token usage? This is because Bedrock has processed this exact request before and can return the same response without using as many tokens as the initial processing required.

## Changing your Bedrock prompt

In [10]:
system_prompt = """
 You are a pet expert that will tell the user what type of pet they have based on a description. You'll keep your responses short and generalized.
 Here are examples:

 Description: My pet barks and has four legs.
 Summary: Your pet is a dog.

 Review: My pet meows and uses a litter box.
 Summary: Your pet is a cat.
 """

user_input_review = "My pet sings and has wings."

Run the cell `With Caching`, the response should look similar to this:

```
[With Caching] Generated Summary:
Your pet is likely a bird, such as a parrot or canary.
Total Tokens Used: 26
 ```

Run the cell `No Caching`, the response should look similar to this:

```
[No Caching] Generated Summary:
 Your pet is likely a bird, such as a parrot or canary.
 Total Tokens Used (No Cache): 98
 ```

 Why does the cached response use less tokens even with a new request? Using `cache_point = {"cachePoint": {"type": "default"}}` with Bedrock on a new request can still use fewer tokens because the system leverages pre-cached computations and optimizations. Even for new requests, parts of the model's processing might be reused from the cache, reducing the need for fresh token computation. This caching strategy helps minimize redundant processing, leading to lower token usage and cost. Essentially, Bedrock efficiently reuses cached intermediate results, even on the first request.